In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Flatten, Dropout, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [4]:
df = pd.read_csv(r"C:\Users\gayat\Downloads\imdb\IMDB Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [6]:
# Label Encoding: Convert 'positive' to 1 and 'negative' to 0
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [7]:
# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

# Text Preprocessing: Tokenization & Padding
vocab_size = 10000  # Number of unique words to keep
max_length = 200  # Max review length
oov_token = "<OOV>"  # Out of vocabulary token

In [8]:
# Initialize Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform input size
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

# Build the Deep Neural Network Model
model = Sequential([
    Embedding(vocab_size, 128, input_length=max_length),  # Embedding layer
    LSTM(64, return_sequences=True),  # LSTM layer for sequential data
    LSTM(32),  # Another LSTM layer
    Dropout(0.5),  # Dropout to prevent overfitting
    Dense(32, activation='relu'),  # Dense hidden layer
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the Model
history = model.fit(X_train_pad, y_train, epochs=20, batch_size=32, validation_data=(X_test_pad, y_test))

# Evaluate the Model
y_pred_prob = model.predict(X_test_pad)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary labels

C:\Users\gayat\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 276s 215ms/step - accuracy: 0.5161 - loss: 0.6919 - val_accuracy: 0.5039 - val_loss: 0.6931
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1079s 863ms/step - accuracy: 0.5029 - loss: 0.6934 - val_accuracy: 0.5093 - val_loss: 0.6929
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 318s 216ms/step - accuracy: 0.5745 - loss: 0.6653 - val_accuracy: 0.6270 - val_loss: 0.6383
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 310s 205ms/step - accuracy: 0.6266 - loss: 0.6469 - val_accuracy: 0.7603 - val_loss: 0.5167
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 262s 210ms/step - accuracy: 0.8313 - loss: 0.4066 - val_accuracy: 0.8510 - val_loss: 0.3601
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41113s 33s/step - accuracy: 0.8977 - loss: 0.2719 - val_accuracy: 0.8810 - val_loss: 0.2880
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 127s 102ms/step - accuracy: 0.9287 - loss: 0.2015 - val_accuracy: 0.8781 - val_loss: 0.3009
Epoch 8/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 100s 80ms/step - ac

In [10]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

# Analyze Misclassifications
misclassified_idx = np.where(y_pred.flatten() != y_test.values)[0]
print(f"Number of Misclassified Samples: {len(misclassified_idx)}")
print("\nSome Misclassified Reviews:")
for i in misclassified_idx[:5]:  # Show 5 misclassified reviews
    print("\nReview:", X_test.iloc[i])
    print("True Sentiment:", "Positive" if y_test.iloc[i] == 1 else "Negative")
    print("Predicted Sentiment:", "Positive" if y_pred[i] == 1 else "Negative")

Test Accuracy: 0.8576
Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.85      0.86      4961
           1       0.86      0.86      0.86      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

Number of Misclassified Samples: 1424

Some Misclassified Reviews:

Review: I really liked this Summerslam due to the look of the arena, the curtains and just the look overall was interesting to me for some reason. Anyways, this could have been one of the best Summerslam's ever if the WWF didn't have Lex Luger in the main event against Yokozuna, now for it's time it was ok to have a huge fat man vs a strong man but I'm glad times have changed. It was a terrible main event just like every match Luger is in is terrible. Other matches on the card were Razor Ramon vs Ted Dibiase, Steiner Brothers vs Heavenly Bodies, Shawn M